##### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2022 Semester 1

## Assignment 2: Sentiment Classification of Tweets

In [1]:
def zero_R(train_data, k_fold):
    print("zero_R: ")
    all_class = train_data[['sentiment']].values
    class_list = []
    for i in all_class:
        if i[0] == 'negative':
            class_list.append(2)
        if i[0] == 'positive':
            class_list.append(1)
        if i[0] == 'neutral':
            class_list.append(0)
    all_text = train_data[['text']].values
    kf = KFold(n_splits=k_fold,random_state=None)
    acc_list = []
    recall_list = []
    preci_list = []
    f1_list = []
    fold = 1
    # split data using 10-fold cross validation 
    for train_index , test_index in kf.split(all_text):
        m = 0
        X_train = []
        X_test = []
        y_train = []
        y_test = []
        for i in range(len(train_index)):
            X_train.append(all_text[train_index[i]])
            y_train.append(class_list[train_index[i]])
        for i in range(len(test_index)):
            X_test.append(all_text[test_index[i]])
            y_test.append(class_list[test_index[i]])
            
        #count the majority of the class in each traing dataset
        label_count = Counter(y_train)
        major_label = label_count.most_common(1)[0][0]
        label = np.full((len(test_index), 1), major_label, dtype = int)
        predict_label = []
        for i in range(len(label)):
            predict_label.append(label[i][0])
        confusion_matrix = metrics.confusion_matrix(y_test, predict_label, labels = [0,1,2])
        accuracy = accuracy_score(y_test, predict_label)
        precision = precision_score(y_test, predict_label, average = None,zero_division=1)
        recall = recall_score(y_test, predict_label, average = None,zero_division=1)
        f1_score = (2*precision*recall)/(recall+precision)
        acc_list.append(accuracy)
        preci_list.append(precision)
        for i in precision:
            if i == 1.0:
                m = 1
        #print result
        recall_list.append(recall)
        f1_list.append(f1_score)
        print("fold "+str(fold))
        print(confusion_matrix)
        print("Accuracy: "+str(accuracy))
        print("Precision: "+str(precision))
        print("Recall: "+str(recall))
        print("F1 score: "+str(f1_score))
        print('\n')
        fold += 1   
    print('Average Accuracy with '+ str(k_fold) +' fold: '+ str(sum(acc_list)/k_fold))
    if m != 1:
        print('Average Precision with '+ str(k_fold) +' fold: '+ str(sum(preci_list)/k_fold))
        print('Average Recall with '+ str(k_fold) +' fold: '+ str(sum(recall_list)/k_fold))
        print('Average F1 score with '+ str(k_fold) +' fold: '+ str(sum(f1_list)/k_fold))

In [2]:
# split data using 10-fold cross validation
def data_split(clean_text, class_label, k_fold):
    all_text = clean_text
    all_class = class_label
    kf = KFold(n_splits=k_fold,random_state=None)
    X_train_list = []
    X_test_list = []
    y_train_list = []
    y_test_list = []
    for train_index , test_index in kf.split(all_text):
        m = 0
        X_train = []
        X_test = []
        y_train = []
        y_test = []
        for i in range(len(train_index)):
            X_train.append(all_text[train_index[i]])
            y_train.append(all_class[train_index[i]])
        for i in range(len(test_index)):
            X_test.append(all_text[test_index[i]])
            y_test.append(all_class[test_index[i]])
        X_train_list.append(X_train)
        X_test_list.append(X_test)
        y_train_list.append(y_train)
        y_test_list.append(y_test)
    return X_train_list,X_test_list,y_train_list,y_test_list

In [3]:
#import all needed resources
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import re
import nltk
import numpy as np
from sklearn import svm
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import metrics
from nltk.corpus import stopwords
from sklearn.model_selection import KFold
from collections import Counter
train_data = pd.read_csv("Train.csv", sep=',')
test_data = pd.read_csv("Test.csv", sep=',')

#implement zero R baseline
zero_R(train_data,10)

zero_R: 
fold 1
[[1231    0    0]
 [ 556    0    0]
 [ 394    0    0]]
Accuracy: 0.5644199908298946
Precision: [0.56441999 1.         1.        ]
Recall: [1. 0. 0.]
F1 score: [0.72157093 0.         0.        ]


fold 2
[[1215    0    0]
 [ 563    0    0]
 [ 403    0    0]]
Accuracy: 0.5570839064649243
Precision: [0.55708391 1.         1.        ]
Recall: [1. 0. 0.]
F1 score: [0.7155477 0.        0.       ]


fold 3
[[1195    0    0]
 [ 574    0    0]
 [ 411    0    0]]
Accuracy: 0.5481651376146789
Precision: [0.54816514 1.         1.        ]
Recall: [1. 0. 0.]
F1 score: [0.70814815 0.         0.        ]


fold 4
[[1179    0    0]
 [ 583    0    0]
 [ 418    0    0]]
Accuracy: 0.5408256880733945
Precision: [0.54082569 1.         1.        ]
Recall: [1. 0. 0.]
F1 score: [0.70199464 0.         0.        ]


fold 5
[[1185    0    0]
 [ 613    0    0]
 [ 382    0    0]]
Accuracy: 0.5435779816513762
Precision: [0.54357798 1.         1.        ]
Recall: [1. 0. 0.]
F1 score: [0.70430906 0.  

In [4]:
all_text = train_data[['text']].values
y = [x[0] for x in train_data[['sentiment']].values]
clean_text_list = []
for x in all_text:
    text = re.sub("\S*https?:\S*", " ", x[0])
    text = re.sub('[^A-Za-z]+', ' ', text)
    text = re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', text)
    text = text.lower()
    tokens = word_tokenize(text)
    text = (" ").join(ele for ele in tokens if ele.lower() not in stopwords.words('english'))
    clean_text_list.append(text)
clean_text = clean_text_list

In [5]:
k_fold = 10
X_train_list,X_test_list,y_train_list,y_test_list = data_split(clean_text, y, k_fold)

In [6]:
# define k frequent feature
# if select is 1 then select k best feature before fitting the model
max_features = 1000
print('Bag of Words')
print('Multinomial naive bayes')
select = 0
acc_list_multi = []
macro_list_multi = []
for i in range(k_fold):
    X_train = X_train_list[i]
    X_test = X_test_list[i]
    y_train = y_train_list[i]
    y_test = y_test_list[i]
    if select == 1:
        bow = CountVectorizer()
        X_train = bow.fit_transform(X_train)
        X_test = bow.transform(X_test)
        selector = SelectKBest(chi2, k=5000)
        X_train = selector.fit_transform(X_train, y_train)
        X_test = selector.transform(X_test)
    else:
        bow = CountVectorizer(max_features = max_features)
        X_train = bow.fit_transform(X_train)
        X_test = bow.transform(X_test)
        
    # fit model
    classifier = MultinomialNB(alpha = 1)
    classifier.fit(X_train, y_train)
    from sklearn.metrics import accuracy_score, classification_report
    y_pred = classifier.predict(X_test)
    classification_report = classification_report(y_test, y_pred,digits = 4)
    acc_list_multi.append(accuracy_score(y_test, y_pred))
    macro_list_multi.append(f1_score(y_test, y_pred, average='macro'))
    confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
    # print result
    print('Fold '+str(i+1))
    print(confusion_matrix)
    print('\nClassification Report')
    print('======================================================')
    print('\n', classification_report)
print('Average Accuracy with '+ str(k_fold) +' Fold: '+ str(sum(acc_list_multi)/k_fold))
print('Average Macro f1 score with '+ str(k_fold) +' Fold: '+ str(sum(macro_list_multi)/k_fold))
#max_features = 1000 average accuracy = 0.6093
#best max_features = 2400 average accuracy = 0.6164
#k = 1000 average accuracy = 0.6325
#k = 2000 average accuracy = 0.6273
#k = 3000 average accuracy = 0.6291
#k = 5000 average accuracy = 0.6355

Bag of Words
Multinomial naive bayes
Fold 1
[[194 171  29]
 [190 847 194]
 [ 28 206 322]]

Classification Report

               precision    recall  f1-score   support

    negative     0.4709    0.4924    0.4814       394
     neutral     0.6920    0.6881    0.6900      1231
    positive     0.5908    0.5791    0.5849       556

    accuracy                         0.6249      2181
   macro avg     0.5846    0.5865    0.5854      2181
weighted avg     0.6263    0.6249    0.6255      2181

Fold 2
[[213 168  22]
 [212 808 195]
 [ 25 236 302]]

Classification Report

               precision    recall  f1-score   support

    negative     0.4733    0.5285    0.4994       403
     neutral     0.6667    0.6650    0.6658      1215
    positive     0.5819    0.5364    0.5582       563

    accuracy                         0.6066      2181
   macro avg     0.5740    0.5767    0.5745      2181
weighted avg     0.6091    0.6066    0.6073      2181

Fold 3
[[229 152  30]
 [175 813 207]
 [ 32 22

In [7]:
print('Bag of Words')
print('SVM')
max_features = 1000
# select best feature if select = 1
select = 0
acc_list_svm = []
macro_list_svm = []
for i in range(k_fold):
    X_train = X_train_list[i]
    X_test = X_test_list[i]
    y_train = y_train_list[i]
    y_test = y_test_list[i]
    if select == 1:
        bow = CountVectorizer()
        X_train = bow.fit_transform(X_train)
        X_test = bow.transform(X_test)
        selector = SelectKBest(chi2, k=2000)
        X_train = selector.fit_transform(X_train, y_train)
        X_test = selector.transform(X_test)
    else:
        bow = CountVectorizer(max_features = max_features)
        X_train = bow.fit_transform(X_train)
        X_test = bow.transform(X_test)
    classifier = svm.SVC(C=1.0,kernel = 'linear',decision_function_shape='ovr')
    classifier.fit(X_train, y_train)
    from sklearn.metrics import accuracy_score, classification_report
    y_pred = classifier.predict(X_test)
    classification_report = classification_report(y_test, y_pred,digits = 4)
    acc_list_svm.append(accuracy_score(y_test, y_pred))
    macro_list_svm.append(f1_score(y_test, y_pred, average='macro'))
    confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
    print('Fold '+str(i+1))
    print(confusion_matrix)
    print('\nClassification Report')
    print('======================================================')
    print('\n', classification_report)
print('Average Accuracy with '+ str(k_fold) +' Fold: '+ str(sum(acc_list_svm)/k_fold))
print('Average Macro f1 score with '+ str(k_fold) +' Fold: '+ str(sum(macro_list_svm)/k_fold))
#best max_features = 1000 average accuracy = 0.6510
#max_features = 2400 average accuracy = 0.6482 
#max_features = 3000 average accuracy = 0.6441
#k = 5000 average accuracy = 0.6626
#k = 3000 average accuracy = 0.6624
#k = 2000 average accuracy = 0.6642
#k = 1000 average accuracy = 0.6581

Bag of Words
SVM
Fold 1
[[  89  287   18]
 [  47 1125   59]
 [  15  297  244]]

Classification Report

               precision    recall  f1-score   support

    negative     0.5894    0.2259    0.3266       394
     neutral     0.6583    0.9139    0.7653      1231
    positive     0.7601    0.4388    0.5564       556

    accuracy                         0.6685      2181
   macro avg     0.6693    0.5262    0.5495      2181
weighted avg     0.6718    0.6685    0.6328      2181

Fold 2
[[ 100  293   10]
 [  46 1079   90]
 [   6  333  224]]

Classification Report

               precision    recall  f1-score   support

    negative     0.6579    0.2481    0.3604       403
     neutral     0.6328    0.8881    0.7390      1215
    positive     0.6914    0.3979    0.5051       563

    accuracy                         0.6433      2181
   macro avg     0.6607    0.5114    0.5348      2181
weighted avg     0.6526    0.6433    0.6087      2181

Fold 3
[[  95  298   18]
 [  49 1061   85]
 [  

In [8]:
print('tf')
print('multinomial naive bayes')
max_features = 1000
select = 0
acc_list = []
macro_list = []
for i in range(k_fold):
    X_train = X_train_list[i]
    X_test = X_test_list[i]
    y_train = y_train_list[i]
    y_test = y_test_list[i]
    if select == 1:
        td = TfidfVectorizer()
        X_train = td.fit_transform(X_train)
        X_test = td.transform(X_test)
        selector = SelectKBest(chi2, k=200)
        X_train = selector.fit_transform(X_train, y_train)
        X_test = selector.transform(X_test)
    else:
        td = TfidfVectorizer(max_features = max_features)
        X_train = td.fit_transform(X_train)
        X_test = td.transform(X_test)
    classifier = MultinomialNB(alpha = 1)
    classifier.fit(X_train, y_train)
    from sklearn.metrics import accuracy_score, classification_report
    y_pred = classifier.predict(X_test)
    classification_report = classification_report(y_test, y_pred,digits = 4)
    confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
    acc_list.append(accuracy_score(y_test, y_pred))
    macro_list.append(f1_score(y_test, y_pred, average='macro'))
    print('Fold '+str(i+1))
    print(confusion_matrix)
    print('Classification Report')
    print('======================================================')
    print('\n', classification_report)
print('Average Accuracy with '+ str(k_fold) +' Fold: '+ str(sum(acc_list)/k_fold))
print('Average Macro f1 score with '+ str(k_fold) +' Fold: '+ str(sum(macro_list)/k_fold))
#max_features = 1000 average accuracy = 0.6349
#Best max_features = 2400 average accuracy = 0.6429
#max_feature = 3000 average accuracy = 0.6416
#k = 1000 average accuracy = 0.6325
#k = 2000 average accuracy = 0.6273
#k = 3000 average accuracy = 0.6291
#k = 5000 average accuracy = 0.6355

tf
multinomial naive bayes
Fold 1
[[  76  304   14]
 [  47 1122   62]
 [   3  365  188]]
Classification Report

               precision    recall  f1-score   support

    negative     0.6032    0.1929    0.2923       394
     neutral     0.6265    0.9115    0.7426      1231
    positive     0.7121    0.3381    0.4585       556

    accuracy                         0.6355      2181
   macro avg     0.6473    0.4808    0.4978      2181
weighted avg     0.6441    0.6355    0.5888      2181

Fold 2
[[  84  312    7]
 [  45 1120   50]
 [   3  380  180]]
Classification Report

               precision    recall  f1-score   support

    negative     0.6364    0.2084    0.3140       403
     neutral     0.6181    0.9218    0.7400      1215
    positive     0.7595    0.3197    0.4500       563

    accuracy                         0.6346      2181
   macro avg     0.6713    0.4833    0.5013      2181
weighted avg     0.6580    0.6346    0.5864      2181

Fold 3
[[  81  322    8]
 [  44 1097   

In [9]:
print('tf')
print('SVM')
max_features = 3000
select = 0
acc_list_svm = []
macro_list_svm = []
for i in range(k_fold):
    X_train = X_train_list[i]
    X_test = X_test_list[i]
    y_train = y_train_list[i]
    y_test = y_test_list[i]
    if select == 1:
        td = TfidfVectorizer()
        X_train = td.fit_transform(X_train)
        X_test = td.transform(X_test)
        selector = SelectKBest(chi2, k=5000)
        X_train = selector.fit_transform(X_train, y_train)
        X_test = selector.transform(X_test)
    else:
        td = TfidfVectorizer(max_features = max_features)
        X_train = td.fit_transform(X_train)
        X_test = td.transform(X_test)
    classifier = svm.SVC(C=1.0,kernel = 'linear',decision_function_shape='ovr')
    classifier.fit(X_train, y_train)
    from sklearn.metrics import accuracy_score, classification_report
    y_pred = classifier.predict(X_test)
    classification_report = classification_report(y_test, y_pred,digits = 4)
    acc_list_svm.append(accuracy_score(y_test, y_pred))
    macro_list_svm.append(f1_score(y_test, y_pred, average='macro'))
    confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
    print('Fold '+str(i+1))
    print(confusion_matrix)
    print('\nClassification Report')
    print('======================================================')
    print('\n', classification_report)
print('Average Accuracy with '+ str(k_fold) +' Fold: '+ str(sum(acc_list_svm)/k_fold))
print('Average Macro f1 score with '+ str(k_fold) +' Fold: '+ str(sum(macro_list_svm)/k_fold))
#max_features = 1000 average accuracy = 0.6509
#max_features = 2400 average accuracy = 0.6622
#max_features = 3000 average accuracy = 0.6643
#k = 1000 average accuracy = 0.6531
#k = 2000 average accuracy = 0.6577
#k = 3000 average accuracy = 0.6592
#k = 5000 average accuracy = 0.6612

tf
SVM
Fold 1
[[ 110  267   17]
 [  61 1098   72]
 [  13  291  252]]

Classification Report

               precision    recall  f1-score   support

    negative     0.5978    0.2792    0.3806       394
     neutral     0.6630    0.8920    0.7607      1231
    positive     0.7390    0.4532    0.5619       556

    accuracy                         0.6694      2181
   macro avg     0.6666    0.5415    0.5677      2181
weighted avg     0.6706    0.6694    0.6413      2181

Fold 2
[[ 124  272    7]
 [  47 1089   79]
 [   2  336  225]]

Classification Report

               precision    recall  f1-score   support

    negative     0.7168    0.3077    0.4306       403
     neutral     0.6417    0.8963    0.7479      1215
    positive     0.7235    0.3996    0.5149       563

    accuracy                         0.6593      2181
   macro avg     0.6940    0.5345    0.5645      2181
weighted avg     0.6767    0.6593    0.6291      2181

Fold 3
[[ 110  285   16]
 [  54 1065   76]
 [   8  322  2

In [13]:
#predict test.csv
max_features = 3000
X_train = clean_text
all_text = test_data[['text']].values
y_train = [x[0] for x in train_data[['sentiment']].values]
clean_text_test = []
for x in all_text:
    text = re.sub("\S*https?:\S*", " ", x[0])
    text = re.sub('[^A-Za-z]+', ' ', text)
    text = re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', text)
    text = text.lower()
    tokens = word_tokenize(text)
    text = (" ").join(ele for ele in tokens if ele.lower() not in stopwords.words('english'))
    clean_text_test.append(text)
X_test = clean_text_test
td = TfidfVectorizer(max_features = max_features)
X_train = td.fit_transform(X_train)
X_test = td.transform(X_test)
classifier = svm.SVC(C=1.0,kernel = 'linear',decision_function_shape='ovr')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
predict_csv = pd.read_csv("Test.csv", sep=',')
predict_csv.drop('text', inplace=True, axis=1)
predict_csv['sentiment'] = y_pred
predict_csv.to_csv('prediction.csv', index = False)